In [10]:
import cobra
import os 
import sys
import pandas as pd
import numpy as np

In [3]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils
import ecpy

In [38]:
import importlib
importlib.reload(utils)
importlib.reload(ecpy)

<module 'ecpy' from '/Users/gangl/Documents/GitHub/Halo-GEM/ecpy/ecpy.py'>

#### 1. Load the GEM model

In [35]:
model_file = '../../../ModelFiles/json/Halo_GEM_v1.json'
model = cobra.io.load_json_model(model_file)

In [36]:
utils.report_model(model)

Reactions: 1932
Metabolites: 2015


In [39]:
utils.test_biomass_production(model,'Biomass_v1')

<Solution 5.748 at 0x132c787160>


#### 2. Convert to irreversible model

In [30]:
irrModel = ecpy.convertToIrrev(model)

In [31]:
utils.report_model(irrModel)

Reactions: 2327
Metabolites: 2015


In [40]:
utils.test_biomass_production(irrModel,'Biomass_v1')

<Solution 5.748 at 0x132c79e240>


#### 3. Add enzymes into model: eModel, need kcat values

##### 3.1 prepare a dictionary with (enz_id,rxn_id) as keys and kcasts as values
In this reconstruction, kcats for reactions were matched only at the reaction level, meaning that all iso-enzymes that catalyze the same reaction have the same kcat.

In [11]:
df_enz_kcat = pd.read_csv('../Results/mapped_kcats.csv',index_col=0)
# In this file, all kcats are in the unit of 1/s and log10 transformed

In [12]:
df_enz_kcat.head(5)

,log10_kcat_mean,log10_kcat_std
CHOLD,1.026202,1.026202
DEOXYRIBOSE-P-ALD-RXN,1.108909,1.108909
RXN-6321,1.223489,1.223489
GLYOHMETRANS-RXN,1.435783,1.435783
GLYOHMETRANS-RXN_REV,0.698970,1.000000


In [14]:
kcat_dict = ecpy.prepare_kcats_dict(irrModel,df_enz_kcat)
# In this dictionary, all kcats are in the unit of 1/h 

In [15]:
len(kcat_dict)

2391

##### 3.2 add enzymes into reactions

In [16]:
eModel = ecpy.convertToEnzymeModel(irrModel,kcat_dict)

In [17]:
utils.report_model(eModel)

Reactions: 4412
Metabolites: 3488


In [41]:
utils.test_biomass_production(eModel,'Biomass_v1')

<Solution 5.748 at 0x132a782470>


#### 4. add protein pool

In [43]:
df_enz_mw = pd.read_csv('../Results/enzyme_MWs.csv',index_col=0)
df_enz_mw.head(5)

,mw
TD01GL003263,28.861369
TD01GL001719,31.849069
TD01GL002685,30.097315
TD01GL001672,28.361097
TD01GL000345,28.302825


In [45]:
MWs = {ind:df_enz_mw.loc[ind,'mw'] for ind in df_enz_mw.index}

In [47]:
ecModel = ecpy.constrainPool(eModel,MWs,list(MWs.keys()),0.018)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmptaky356y.lp
Reading time = 0.03 seconds
: 3488 rows, 8824 columns, 34138 nonzeros


In [52]:
rxn_pool = ecModel.reactions.get_by_id('prot_pool_exchange')
print(rxn_pool.upper_bound)

0.018


In [53]:
rxn_pool.upper_bound = 0.098
utils.test_biomass_production(ecModel,'Biomass_v1')

<Solution 0.058 at 0x1327b816a0>
